In [62]:
!pip3 install transformers
!pip3 install datasets
!pip3 install pytesseract
!pip3 install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 65.9 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 140 kB 69.9 MB/s 
     |████████████████████████████████| 1.1 MB 71.9 MB/s 
     |████████████████████████████████| 127 kB 75.7 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |█████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 5.0 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=e40620528d957b792c777c85e47e1502d825389bfc56cae798dcf6794e4999bf
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [63]:
!python -m pip install --upgrade detectron2==0.6 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 579 kB/s 
     |████████████████████████████████| 50 kB 3.1 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 151 kB 46.5 MB/s 
     |████████████████████████████████| 130 kB 67.8 MB/s 
     |████████████████████████████████| 843 kB 65.2 MB/s 
     |████████████████████████████████| 117 kB 74.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=44d91be8ec848657566706c871d9f77d9684312560a7603a0a6f62f989f9668c
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=9eabcfb33b0cee39c6

In [64]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 87 not upgraded.
Need to get 7,168 kB of archives.
After this operation, 7,962 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.2.0 [7,168 kB]
Fetched 7,168 kB in 1s (10.3 MB/s)
debconf: unable to initializ

## Download dataset

In [65]:
import torch
import json
import numpy as np
import pandas as pd
  
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Preparing Dataset

In [66]:
# Dataset: https://www.kaggle.com/datasets/finalepoch/medical-ner

with open('data/annotations.json') as f:
  annotations = json.load(f)['examples']

In [106]:
# Convert the tag_name(s) in IOB-format
# sentence: I have a brain tumour .
# tags: 'O', 'O', 'O', 'B-medical_condition', 'I-medical_condition', 'O'

dataset = []

for idx, data in enumerate(annotations):
  sentence_tokens = data['content'].lower().split(" ")
  labels = ['O']*len(sentence_tokens)
  entities = data['annotations']
  tag_text_pair = {i['value'].lower(): i['tag_name'] for i in entities if i['value'] is not ' '}
  for word_idx, word in enumerate(sentence_tokens):
    if word in list(tag_text_pair.keys()):
      labels[word_idx] = f'B-{tag_text_pair[word]}'
  dataset.append({
                  'sentence_ids': f'sentence: {idx+1}',
                  'sentences_tokens': sentence_tokens,
                  'tags': labels
                  })

In [107]:
df = pd.DataFrame(dataset)
df.head()

,sentence_ids,sentences_tokens,tags
0,sentence: 1,"[while, bismuth, compounds, (pepto-bismol), de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,sentence: 2,"[diarrhea,, also, spelled, diarrhoea,, is, the...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,sentence: 3,"[antiretroviral, therapy, (art), is, recommend...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,sentence: 4,"[the, following, drugs, are, considered, as, d...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-M..."
4,sentence: 5,"[the, goals, of, treatment, are, to, reduce, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [108]:
' '.join(df.sentences_tokens[0])

"while bismuth compounds (pepto-bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] these agents should be used only if bloody diarrhea is not present.[92]\n\ndiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nracecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] it has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [109]:
','.join(df.tags[0])

'O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-Medicine,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-MedicalCondition,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-MedicalCondition,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-MedicalCondition,O,O,O,O,O,O,O,O,O,O,B-MedicalCondition,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-MedicalCondition,O,O'

In [110]:
first_row = df['tags'][0].copy()
print(first_row)

all_tags = first_row
for i in df['tags'][1:]:
  all_tags.extend(i)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Medicine', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MedicalCondition', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MedicalCondition', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MedicalCondition', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MedicalCondition', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MedicalCondition', 'O', 'O']


In [111]:
print('Total of entities: {} \nNumber of unique entities: {}'.format(len(all_tags), len(set(all_tags))))
print('Unique entities: {}'.format(set(all_tags)))

unique_entities = list(set(all_tags))
print(unique_entities)

Total of entities: 3114 
Number of unique entities: 4
Unique entities: {'O', 'B-Pathogen', 'B-Medicine', 'B-MedicalCondition'}
['O', 'B-Pathogen', 'B-Medicine', 'B-MedicalCondition']


In [112]:
label2id = {label:id for id, label in enumerate(unique_entities)}
id2label = {id:label for id, label in enumerate(unique_entities)}
label2id

{'B-MedicalCondition': 3, 'B-Medicine': 2, 'B-Pathogen': 1, 'O': 0}

**Hyperparameters**

In [162]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 1e-5
MAX_GRAD_NORM = 10 # What is the use of this hyperparameter?
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

**Helper function**

In [163]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
  """
  Word piece tokenization makes it difficult to match word labels
  back up with individual word pieces. This function tokenizes each
  word one at a time so that it is easier to preserve the correct
  label for each subword. It is, ofcourse, a bit slower in processing
  time, but it will help our model achieve higher accuracy
  """
  tokenized_sentence = []
  labels = []

  # sentence = sentence.strip()

  for word, label in zip(sentence, text_labels):

    # Tokenize the word and count # of subwords the word is broken into
    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

## Creating DataLoaders

In [164]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.len = len(dataframe)
    self.data = dataframe
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __getitem__(self, index):
    # step 1: tokenize (and adapt corresponding labels)
    # print("# step 1: tokenize (and adapt corresponding labels)")
    sentence = self.data.sentences_tokens[index]
    # print("# of sentence: {}".format(len(sentence)))
    word_labels = self.data.tags[index]
    # print("# of labels: {}".format(len(word_labels)))

    tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

    # step 2: add special tokens (and corresponding labels)
    # print("# step 2: add special tokens (and corresponding labels)")
    tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"]
    labels.insert(0, "O") # add outside label for [CLS] token
    labels.insert(-1, "O") # add outside label for [SEP] token
    # print("Labels: {}".format(labels[1]))
    # print("Labels: {}".format(labels[0][1]))

    # step 3: truncating/padding
    # print("# step 3: truncating/padding")
    maxlen = self.max_len

    if len(tokenized_sentence) > maxlen:
      # truncate
      tokenized_sentence = tokenized_sentence[:maxlen]
      labels = labels[:maxlen]
    else:
      # pad
      tokenized_sentence = tokenized_sentence + ['[PAD]' for _ in range(maxlen - len(tokenized_sentence))]
      labels = labels + ['O' for _ in range(maxlen - len(labels))]

    # print("Labels: {}".format(labels))

    # step 4: obtain the attention mask
    attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

    # step 5: convert tokens to input ids
    # print("# step 5: convert tokens to input ids")
    ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

    label_ids = [label2id[label] for label in labels]
    # print("Labels: {}".format(label_ids))

    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(attn_mask, dtype=torch.long),
        'targets': torch.tensor(label_ids, dtype=torch.long)
    }

  def __len__(self):
    return self.len

In [165]:
train_size = 0.75
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)


FULL Dataset: (31, 3)
TRAIN Dataset: (23, 3)
TEST Dataset: (8, 3)


In [166]:
training_set[0]

{'ids': tensor([  101,  4973,  1997,  2691,  2529,  7870,  3303,  2011, 18191,  2421,
          1996,  2691,  3147,  1010, 24442,  1010,  7975,  6873,  2595,  1010,
          1998,  3147, 14699,  2015,  1012,  2116,  3809,  7870,  2107,  2004,
         10958, 20536,  1010,  1041, 24290,  7865,  4295,  1010,  8387,  1006,
          9820,  1007,  1010, 20704,  2937, 24442,  1010,  1998, 18906,  2015,
          2024,  3303,  2011, 18191,  1012,  1996,  5816,  3754,  1997, 18191,
          2000,  3426,  4295,  2003,  2649,  1999,  3408,  1997,  6819,  6820,
         22717,  1012,  2060,  7870,  2024,  2104,  4812,  2000,  7523,  2065,
          2027,  2031,  1037,  7865,  2004,  1996,  6187, 10383,  6024,  4005,
          1010,  2107,  2004,  1996,  2825,  4434,  2090,  2529,  2014, 10374,
         23350,  1020,  1006,  1044,  2232,  2615,  2575,  1007,  1998, 23130,
          7870,  2107,  2004,  3674,  8040,  3917, 12650,  1998, 11888, 16342,
          8715,  1012,  1031, 16528,  1033,  

In [167]:
training_set[0]['ids']

tensor([  101,  4973,  1997,  2691,  2529,  7870,  3303,  2011, 18191,  2421,
         1996,  2691,  3147,  1010, 24442,  1010,  7975,  6873,  2595,  1010,
         1998,  3147, 14699,  2015,  1012,  2116,  3809,  7870,  2107,  2004,
        10958, 20536,  1010,  1041, 24290,  7865,  4295,  1010,  8387,  1006,
         9820,  1007,  1010, 20704,  2937, 24442,  1010,  1998, 18906,  2015,
         2024,  3303,  2011, 18191,  1012,  1996,  5816,  3754,  1997, 18191,
         2000,  3426,  4295,  2003,  2649,  1999,  3408,  1997,  6819,  6820,
        22717,  1012,  2060,  7870,  2024,  2104,  4812,  2000,  7523,  2065,
         2027,  2031,  1037,  7865,  2004,  1996,  6187, 10383,  6024,  4005,
         1010,  2107,  2004,  1996,  2825,  4434,  2090,  2529,  2014, 10374,
        23350,  1020,  1006,  1044,  2232,  2615,  2575,  1007,  1998, 23130,
         7870,  2107,  2004,  3674,  8040,  3917, 12650,  1998, 11888, 16342,
         8715,  1012,  1031, 16528,  1033,  2045,  2003,  6704])

In [168]:
# print the first 50 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:50]), tokenizer.convert_ids_to_tokens(training_set[0]["targets"][:50])):
  # print('{0:10} {1}'.format(token, id2label[label]))
  print(token, label)


[CLS] [PAD]
examples [PAD]
of [PAD]
common [PAD]
human [PAD]
diseases [PAD]
caused [PAD]
by [PAD]
viruses [PAD]
include [PAD]
the [PAD]
common [PAD]
cold [PAD]
, [PAD]
influenza [PAD]
, [PAD]
chicken [PAD]
##po [PAD]
##x [PAD]
, [PAD]
and [PAD]
cold [PAD]
sore [PAD]
##s [PAD]
. [PAD]
many [PAD]
serious [PAD]
diseases [PAD]
such [PAD]
as [PAD]
ra [PAD]
##bies [PAD]
, [PAD]
e [PAD]
##bola [PAD]
virus [PAD]
disease [PAD]
, [PAD]
aids [PAD]
( [PAD]
hiv [PAD]
) [PAD]
, [PAD]
av [PAD]
##ian [PAD]
influenza [PAD]
, [PAD]
and [PAD]
sar [PAD]
##s [PAD]


## Defining the model

In [169]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [170]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [171]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.3559, device='cuda:0', grad_fn=<NllLossBackward0>)

In [172]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 4])

In [173]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

## Model Training

In [174]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per {idx} training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [175]:
EPOCHS = 1000 # Overwriting EPOCHS mentioned in hyperparameter initialization

for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 0 training steps: 1.4575726985931396
Training loss epoch: 0.8903892834981283
Training accuracy epoch: 0.7618631363385907
Training epoch: 2
Training loss per 0 training steps: 0.5635035037994385
Training loss epoch: 0.3310677744448185
Training accuracy epoch: 0.9445771510423354
Training epoch: 3
Training loss per 0 training steps: 0.1783135086297989
Training loss epoch: 0.24367605770627657
Training accuracy epoch: 0.9456320831720566
Training epoch: 4
Training loss per 0 training steps: 0.4173550605773926
Training loss epoch: 0.212019138969481
Training accuracy epoch: 0.9473436069802929
Training epoch: 5
Training loss per 0 training steps: 0.03569668158888817
Training loss epoch: 0.19982089567929506
Training accuracy epoch: 0.9475746967506972
Training epoch: 6
Training loss per 0 training steps: 0.1393236517906189
Training loss epoch: 0.18896524732311568
Training accuracy epoch: 0.9422693425462597
Training epoch: 7
Training loss per 0 training steps: 0

## Model Validation

In [176]:
def validation(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per {idx} evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [177]:
labels, predictions = validation(model, testing_loader)

Validation loss per 0 evaluation steps: 1.8650262063601986e-05
Validation Loss: 0.48969188373087036
Validation Accuracy: 0.9354299532312925


In [178]:
from seqeval.metrics import classification_report
# print(labels)
# print(predictions)

print(classification_report([labels], [predictions]))

                  precision    recall  f1-score   support

MedicalCondition       0.00      0.00      0.00         1
        Medicine       0.60      0.44      0.51        57
        Pathogen       0.50      0.45      0.48        11

       micro avg       0.58      0.43      0.50        69
       macro avg       0.37      0.30      0.33        69
    weighted avg       0.57      0.43      0.49        69



/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Inference

In [180]:
sentence = "I have malaria and my friend has a common cold."

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

i have malaria and my friend has a common cold .
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
